In [ ]:
#| default_exp js

# Javascript and Formatting Components

To expedite fast development, FastHTML comes with several built-in Javascript and formatting components.

In [ ]:
#| exporti
import re
from fastcore.utils import *
from fastcore.xml import to_xml
from fasthtml.xtend import Script,jsd,Style,Link

In [ ]:
#| export
def light_media(css):
    "Render light media for day mode views"
    return Style('@media (prefers-color-scheme: light) {%s}' %css)

In [ ]:
assert to_xml(light_media('.body {color: green;}')) == '<style>@media (prefers-color-scheme: light) {.body {color: green;}}</style>\n'
light_media('.body {color: green;}')

```html
<style>@media (prefers-color-scheme: light) {.body {color: green;}}</style>

```

In [ ]:
#| export
def  dark_media(css): return Style('@media (prefers-color-scheme:  dark) {%s}' %css)

In [ ]:
assert to_xml(dark_media('.body {color: white;}')) == '<style>@media (prefers-color-scheme:  dark) {.body {color: white;}}</style>\n'
dark_media('.body {color: white;}')

```html
<style>@media (prefers-color-scheme:  dark) {.body {color: white;}}</style>

```

In [ ]:
#| export
marked_imp = """import { marked } from "https://cdn.jsdelivr.net/npm/marked/lib/marked.esm.js";
    import { proc_htmx } from "https://cdn.jsdelivr.net/gh/answerdotai/fasthtml-js/fasthtml.js";
"""
def MarkdownJS(sel='.marked'):
    src = "proc_htmx('%s', e => e.innerHTML = marked.parse(e.textContent));" % sel
    return Script(marked_imp+src, type='module')

In [ ]:
assert to_xml(MarkdownJS()) == '<script type="module">import { marked } from "https://cdn.jsdelivr.net/npm/marked/lib/marked.esm.js";\n    import { proc_htmx } from "https://cdn.jsdelivr.net/gh/answerdotai/fasthtml-js/fasthtml.js";\nproc_htmx(\'.marked\', e => e.innerHTML = marked.parse(e.textContent));</script>\n'
assert to_xml(MarkdownJS('pre')) == '<script type="module">import { marked } from "https://cdn.jsdelivr.net/npm/marked/lib/marked.esm.js";\n    import { proc_htmx } from "https://cdn.jsdelivr.net/gh/answerdotai/fasthtml-js/fasthtml.js";\nproc_htmx(\'pre\', e => e.innerHTML = marked.parse(e.textContent));</script>\n'
MarkdownJS()

```html
<script type="module">import { marked } from "https://cdn.jsdelivr.net/npm/marked/lib/marked.esm.js";
    import { proc_htmx } from "https://cdn.jsdelivr.net/gh/answerdotai/fasthtml-js/fasthtml.js";
proc_htmx('.marked', e => e.innerHTML = marked.parse(e.textContent));</script>

```

In [ ]:
#| export
def KatexMarkdownJS(sel='.marked', inline_delim='$', display_delim='$$', math_envs=None):
    math_envs = math_envs or ['equation', 'align', 'gather', 'multline']
    env_list = ','.join(f"'{env}'" for env in math_envs)

    src = r"""
    import katex from "https://cdn.jsdelivr.net/npm/katex/dist/katex.mjs";
    const renderMath = (tex, displayMode) => { return katex.renderToString(tex, {
            throwOnError: false,
            displayMode: displayMode,
            output: 'html',
            trust: true
        });
    };
    const processLatexEnvironments = (content) => {
        return content.replace(/\\begin{(\w+)}([\s\S]*?)\\end{\1}/g, (match, env, innerContent) => {
            if ([%(env_list)s].includes(env)) { return `%(display_delim)s${match}%(display_delim)s`; }
            return match;
        });
    };
    proc_htmx('%(sel)s', e => {
        let content = processLatexEnvironments(e.textContent);
        // Handle display math (including environments)
        content = content.replace(/%(display_delim)s([\s\S]+?)%(display_delim)s/gm, (_, tex) => renderMath(tex.trim(), true));
        // Handle inline math
        content = content.replace(/(?<!\w)%(inline_delim)s([^%(inline_delim)s\s](?:[^%(inline_delim)s]*[^%(inline_delim)s\s])?)%(inline_delim)s(?!\w)/g, (_, tex) => renderMath(tex.trim(), false));
        e.innerHTML = marked.parse(content);
    });
    """
    format_dict = {
        'env_list': env_list,
        'sel': sel,
        'display_delim': re.escape(display_delim),
        'inline_delim': re.escape(inline_delim)
    }
    formatted_src = src % format_dict
    return (Script(marked_imp + formatted_src, type='module'),
            Link(rel="stylesheet", href="https://cdn.jsdelivr.net/npm/katex@0.16.11/dist/katex.min.css"))

In [ ]:
#| export
def HighlightJS(sel='pre code', langs:str|list|tuple='python', light='atom-one-light', dark='atom-one-dark'):
    src = """
hljs.addPlugin(new CopyButtonPlugin());
hljs.configure({'cssSelector': '%s'});
htmx.onLoad(hljs.highlightAll);""" % sel
    hjs = 'highlightjs','cdn-release', 'build'
    hjc = 'arronhunt'  ,'highlightjs-copy', 'dist'
    if isinstance(langs, str): langs = [langs]
    langjs = [jsd(*hjs, f'languages/{lang}.min.js') for lang in langs]
    return [jsd(*hjs, f'styles/{dark}.css', typ='css', media="(prefers-color-scheme: dark)"),
            jsd(*hjs, f'styles/{light}.css', typ='css', media="(prefers-color-scheme: light)"),
            jsd(*hjs, f'highlight.min.js'),
            jsd(*hjc, 'highlightjs-copy.min.js'),
            jsd(*hjc, 'highlightjs-copy.min.css', typ='css'),
            light_media('.hljs-copy-button {background-color: #2d2b57;}'),
            *langjs, Script(src, type='module')]

In [ ]:
#| export
def SortableJS(sel='.sortable', ghost_class='blue-background-class'):
    src = """
import {Sortable} from 'https://cdn.jsdelivr.net/npm/sortablejs/+esm';
import {proc_htmx} from "https://cdn.jsdelivr.net/gh/answerdotai/fasthtml-js/fasthtml.js";
proc_htmx('%s', Sortable.create);
""" % sel
    return Script(src, type='module')